In [1]:
import numpy as np
import scipy as sp
from statsmodels.stats.proportion import proportion_confint
import scipy

In [2]:
proportion_confint(10,34,method='wilson')

(0.16834630670422424, 0.46168909794714441)

In [3]:
proportion_confint(4,16,method='wilson')

(0.10182067491213043, 0.4949831653550878)

In [4]:
p1_zev = 10/34.0
p2_nezev = 4/16.0
print p1_zev,p2_nezev

0.294117647059 0.25


In [5]:
data_zev = [1]*10 + [0] *24
data_nezev = [1] * 4 + [0] * 12

In [6]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [7]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [8]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [9]:
# 3 вопрос OK
proportions_diff_z_test(proportions_diff_z_stat_ind(data_zev, data_nezev),'greater')

0.37293045872523534

### 4 вопрос

In [10]:
import pandas as pd

In [11]:
bnk_data = pd.read_csv('banknotes.txt',sep='\t',header=0)

In [12]:
bnk_data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [13]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

In [14]:
X_train,X_test,y_train,y_test = train_test_split(bnk_data.drop('real',1),
                                                 bnk_data['real'],random_state = 1,test_size=50)

In [15]:
log_reg1 = LogisticRegression()
log_reg2 = LogisticRegression()

In [16]:
log_reg1.fit(X_train[['X1','X2','X3']],y_train)
log_reg2.fit(X_train[['X4','X5','X6']],y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [18]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [19]:
pred1 = log_reg1.predict(X_test[['X1','X2','X3']])
pred2 = log_reg2.predict(X_test[['X4','X5','X6']])

In [20]:
# 4 вопрос OK
proportions_diff_z_test(proportions_diff_z_stat_rel(abs(pred1 - y_test),
                                                    abs(pred2 - y_test)))

0.0032969384555543435

In [21]:
np.vstack([abs(pred1 - y_test),abs(pred2 - y_test)])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]], dtype=int64)

In [22]:
# 5 вопрос OK
proportions_diff_confint_rel(abs(pred1 - y_test), abs(pred2 - y_test))

(0.059945206279614305, 0.30005479372038568)

### 6 - 7 вопросы 

In [26]:
z = (541.4 - 525) / (100 / np.sqrt(100))
print z
f = scipy.stats.norm.cdf(z)
print f
1 - f

1.64
0.949497416526


0.050502583474103968

In [25]:
z = (541.5 - 525) / (100 / np.sqrt(100))
print z
f = scipy.stats.norm.cdf(z)
print f
1 - f

1.65
0.950528531966


0.049471468033648103